# Конвертируем корпус слов для обучения векторизатора из паркетника в текстовый файл

In [1]:
from pyspark.sql.types import *
from pyspark.sql import SQLContext
import pandas as pd

Читаем паркетник в переменную

In [2]:
sdf = SQLContext(sc).read.parquet("hdfs://e1.getaura.ru/spark-common/tokens1")
sdf.take(3)

[Row(document='магнитный резонансный томография мрт диагностический центр санкт петербург сэкономить 2000 рубль звонить 7 812 385 63 40 c 09 00 00 00 согласие обработка персональный дать мрт мрт голов головной мозг гипофиз глазной орбита пазуха нос мрт позвоночник шейный отдел позвоночник грудной отдел позвоночник поясничный отдел позвоночник копчиковый область крестцовый подвздошный сочленение мрт сустав височный нижнечелюстной сустав плечевой сустав локтевой сустав лучезапястный сустав кисть тазобедренный сустав коленный сустав голеностопный сустав стопа мрт внутренний орган грудной клетка молочный железо орган брюшной полость орган забрюшинный пространство мр холангиография орган малое таз кишечник мрт сосуд сосуд головной мозг сосуд шея подключичный артерия грудной отдел аорта брюшной отдел аорта сосуд конечность мрт мягкий ткань мягкий ткань орган шея ткань конечность мрт тело кт кт голов головной мозг орбита пазуха нос кт позвоночник шейный отдел грудной отдел поясничный крестцов

Рандомно перемешиваем строки

In [24]:
from pyspark.sql.functions import rand 
df = sdf.orderBy(rand())

Вытягиваем из всей массы лишь контент

In [28]:
df.rdd.map(lambda r: r.document).repartition(1)\
            .saveAsTextFile("hdfs://e1.getaura.ru/spark-common/tokens.txt")

Py4JJavaError: An error occurred while calling o352.saveAsTextFile.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 124 in stage 13.0 failed 4 times, most recent failure: Lost task 124.3 in stage 13.0 (TID 1725, r5.getaura.ru, executor 3): java.lang.RuntimeException: Cannot reserve additional contiguous bytes in the vectorized reader (requested = 178279508 bytes). As a workaround, you can disable the vectorized reader by setting spark.sql.parquet.enableVectorizedReader to false.
	at org.apache.spark.sql.execution.vectorized.ColumnVector.throwUnsupportedException(ColumnVector.java:323)
	at org.apache.spark.sql.execution.vectorized.ColumnVector.reserve(ColumnVector.java:308)
	at org.apache.spark.sql.execution.vectorized.ColumnVector.appendBytes(ColumnVector.java:731)
	at org.apache.spark.sql.execution.vectorized.OnHeapColumnVector.putByteArray(OnHeapColumnVector.java:410)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedPlainValuesReader.readBina

In [4]:
# sdf = SQLContext(sc).read.parquet("hdfs://e1.getaura.ru/spark-common/semantic-models/ru_models2/ru_dict_574k.corpora")
# sdf.take(3)

In [13]:
# sdf.rdd.map(lambda r: r.document).saveAsSingleTextFile("hdfs://e1.getaura.ru/spark-common/tokens.txt")

Загружаем файл на кластер

In [1]:
import os
!pip install hdfs
import hdfs
inp = "/spark-common/vectors/german_language_models"
hdfs_url ='http://e1.getaura.ru:50070'
cli = hdfs.InsecureClient(hdfs_url)

In [3]:
os.path.join(os.getcwd(),'religion')

'/spark/Learning_Vectors/Deutche/religion'

In [2]:
cli.download(inp, os.getcwd(), overwrite=True)

    100% |################################| 51kB 2.0MB/s ta 0:00:011
  Running setup.py bdist_wheel for hdfs ... done
  Stored in directory: /root/.cache/pip/wheels/fe/a7/05/23e3699975fc20f8a30e00ac1e515ab8c61168e982abe4ce70
Successfully built hdfs
You are using pip version 9.0.1, however version 19.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


'/spark/Learning_Vectors/Deutche/germ_content1.txt'

Проверяем построчное считывание

In [3]:
from gensim.models.word2vec import LineSentence
corpus = LineSentence("tokens.txt/part-00000")